# importing all libs needed

In [ ]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import numpy as np
import sys, os
import pandas as pd
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from keras.utils import np_utils

# Load csv files and add pixels to X and labels to y Load csv files and add pixels to X and labels to y

In [ ]:
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

df=pd.read_csv('fer2013.csv')

# some informations about data in the .csv file

In [4]:
print(df.info())
print(df["Usage"].value_counts())

print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  35887 non-null  int64 
 1   pixels   35887 non-null  object
 2   Usage    35887 non-null  object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB
None
Training       28709
PublicTest      3589
PrivateTest     3589
Name: Usage, dtype: int64
   emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training


# array that contain y_tain in test_y as label : emotions and the x_train in the x_test as value

In [5]:
X_train,train_y,X_test,test_y=[],[],[],[]

for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
            X_train.append(np.array(val,'float32'))
            train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
            X_test.append(np.array(val,'float32'))
            test_y.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")


In [6]:
num_features = 64
num_labels = 7
batch_size = 64
width, height = 48, 48

In [7]:
X_train = np.array(X_train,'float32')
train_y = np.array(train_y,'float32')
X_test = np.array(X_test,'float32')
test_y = np.array(test_y,'float32')

In [8]:
train_y=np_utils.to_categorical(train_y, num_classes=num_labels)
test_y=np_utils.to_categorical(test_y, num_classes=num_labels)

In [ ]:
#we can t produce
#normalize data from 0 to 1 

In [9]:
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)

X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

# we build the cnn 

In [ ]:
model = Sequential()

### 1st 2D convolution layer (la convolution spatiale sur les images).

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))


#### model.add(BatchNormalization())
#the pooling layer
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

### 2nd 2D convolution layer 

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))

#### model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

### 3th 2D convolution layer 
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
#the pooling layer
#### model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

model.add(Flatten())


### fully connected layer ( FC)

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
#add costumed layer
model.add(Dense(num_labels, activation='softmax'))

model.summary()


# Compliling the model

In [ ]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

# Training the model 

In [13]:
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.fit(X_train, train_y,
          callbacks=[es_callback],
          batch_size=batch_size,
          epochs=25,
          verbose=1,
          validation_data=(X_test, test_y),
          shuffle=True)

Epoch 1/25
449/449 [==============================] - 744s 2s/step - loss: 1.7100 - accuracy: 0.3016 - val_loss: 1.5300 - val_accuracy: 0.3904
Epoch 2/25
449/449 [==============================] - 893s 2s/step - loss: 1.4982 - accuracy: 0.4138 - val_loss: 1.4332 - val_accuracy: 0.4299
Epoch 3/25
449/449 [==============================] - 1025s 2s/step - loss: 1.3973 - accuracy: 0.4548 - val_loss: 1.3010 - val_accuracy: 0.5021
Epoch 4/25
449/449 [==============================] - 941s 2s/step - loss: 1.3351 - accuracy: 0.4824 - val_loss: 1.2692 - val_accuracy: 0.5191
Epoch 5/25
449/449 [==============================] - 948s 2s/step - loss: 1.2920 - accuracy: 0.5019 - val_loss: 1.2559 - val_accuracy: 0.5160
Epoch 6/25
449/449 [==============================] - 851s 2s/step - loss: 1.2537 - accuracy: 0.5190 - val_loss: 1.2134 - val_accuracy: 0.5316
Epoch 7/25
449/449 [==============================] - 770s 2s/step - loss: 1.2298 - accuracy: 0.5254 - val_loss: 1.2140 - val_accuracy: 0.535

# Saving the  model to  use it later on as json and h5

In [14]:
fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")

In [15]:
#----------------------------------------------------------------------

In [8]:
from keras.models import model_from_json
from keras.preprocessing import image

In [ ]:
#---------------------------------------------------------------------

# live cam prediction 

In [3]:
import os
import cv2
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image

#load model
model = model_from_json(open("fer.json", "r").read())
#load weights
model.load_weights('fer.h5')

#casscade classifier for the frontal face (opencv2)

face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')


cap=cv2.VideoCapture(0)

while True:
    # captures frame and returns boolean value and captured image
    ret,test_img=cap.read()
    if not ret:
        continue
    gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

    faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)


    for (x,y,w,h) in faces_detected:
        cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=7)
        roi_gray=gray_img[y:y+w,x:x+h]#cropping region of interest i.e. face area from  image
        roi_gray=cv2.resize(roi_gray,(48,48))
        img_pixels = image.img_to_array(roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        img_pixels /= 255

        predictions = model.predict(img_pixels)

        #find max indexed array
        max_index = np.argmax(predictions[0])

        emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
        predicted_emotion = emotions[max_index]

        cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    resized_img = cv2.resize(test_img, (1000, 700))
    cv2.imshow('Facial emotion analysis ',resized_img)


    #wait until 'q' key is pressed to exit
    if cv2.waitKey(10) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows

<function destroyAllWindows>